# ESRGAN (new arch.) on Colab
Official Github Repo: https://github.com/xinntao/ESRGAN <br>
This notebook was curated by M. Ahabb (Ahabbscience Studio)

In [15]:
!nvidia-smi #recommended gpus are p100 and T4

Thu Aug 19 05:40:04 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    76W / 149W |   4855MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

Clone ESRGAN repo

In [16]:
!git clone https://github.com/xinntao/ESRGAN


fatal: destination path 'ESRGAN' already exists and is not an empty directory.


This is optional if you want to link your google drive to the notebook to add files or pretrained models of your choice from your google drive.

In [17]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


(Optional) Add Files via youtube_dl

In [18]:
from google.colab.patches import cv2_imshow
import cv2
import sys

import argparse
import os
import sys
import time
import re

import numpy as np
import torch
from torch.utils.data import DataLoader





In [19]:
!rm  /content/ESRGAN/LR/*.*

In [20]:
!rm  results/*.*

In [21]:
!ls /content/ESRGAN/LR

Run this cell to upload images

In [22]:
%cd /content/ESRGAN/LR

from google.colab import files
uploaded = files.upload() 
for filename in uploaded.keys():
  print('User uploaded file "{name}" with {length} bytes'.format(name=filename, length=len(uploaded[filename])))

/content/ESRGAN/LR


Saving 2.JPG to 2.JPG
User uploaded file "2.JPG" with 177800 bytes


Run this cell if you want to upload your video. Make sure your video filename contains no spaces, e.g, "my_video" - not "my video". And upload one video per operation.

Download pretrained models

In [23]:
import gdown
print("Downloading pretrained models")
output1 = '/content/ESRGAN/models/RRDB_ESRGAN_x4.pth'
output2 = '/content/ESRGAN/models/RRDB_PSNR_x4.pth'
output3 = '/content/ESRGAN/models/PPON_D.pth'
output4 = '/content/ESRGAN/models/PPON_G.pth'
print ('Downloading RRDB_ESRGAN_x4.pth')
gdown.download('https://drive.google.com/uc?id=1TPrz5QKd8DHHt1k8SRtm6tMiPjz_Qene', output1, quiet=True) 
print ('Downloading RRDB_PSNR_x4.pth')


Configure the code and run the cell to upscale images/video frames

In [24]:
#@title Upscale images/video frames
%cd /content/ESRGAN

import os.path as osp
import glob
import cv2
import numpy as np
import torch
import RRDBNet_arch as arch
import requests
import imageio
import requests
import warnings
warnings.filterwarnings("ignore")
from google.colab import files

Choose_device = "cuda"  #@param ["cuda","cpu"]

model_path = 'models/RRDB_ESRGAN_x4.pth' #@param ['models/RRDB_ESRGAN_x4.pth','models/RRDB_PSNR_x4.pth','models/PPON_G.pth','models/PPON_D.pth']  
device = torch.device(Choose_device) 


test_img_folder = 'LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('results/{:s}.png'.format(base), output)


/content/ESRGAN
Model path models/RRDB_ESRGAN_x4.pth. 
Testing...
1 2


In [25]:
!ls results

2.png


In [ ]:
from IPython.display import Image
Image('results/2.png', width="600")

Make a zip file of your results